In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("../../data/raw/transactions_data.csv")

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,7475327,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,NaN
1,7475328,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,NaN
2,7475329,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,NaN
3,7475331,2010-01-01 00:05:00,430,2860,$200.00,Swipe Transaction,27092,Crown Point,IN,46307.0,4829,NaN
4,7475332,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13305910,23761868,2019-10-31 23:56:00,1718,2379,$1.11,Chip Transaction,86438,West Covina,CA,91792.0,5499,NaN
13305911,23761869,2019-10-31 23:56:00,1766,2066,$12.80,Online Transaction,39261,ONLINE,NaN,NaN,5815,NaN
13305912,23761870,2019-10-31 23:57:00,199,1031,$40.44,Swipe Transaction,2925,Allen,TX,75002.0,4900,NaN
13305913,23761873,2019-10-31 23:58:00,1986,5443,$4.00,Chip Transaction,46284,Daly City,CA,94014.0,5411,NaN


import requests
import time

def min_expires(expires1, expires2):
    m1 = int(expires1.split('/')[0])
    y1 = int(expires1.split('/')[1])
    m2 = int(expires2.split('/')[0])
    y2 = int(expires2.split('/')[1])

    if y1 == y2:
        if m1 < m2:
            return m1
        elif m1 > m2:
            return m2
        else:
            return 'empate'
    elif y1 < y2:
        return y1
    return y2

min = '12/1999'
resul = {}

cont = 0
for client in client_ids:
    print(cont)
    r = requests.get('https://faas-lon1-917a94a7.doserverless.co/api/v1/web/fn-a1f52b59-3551-477f-b8f3-de612fbf2769/default/cards-data', params = {'client_id': client})
    r = r.json()
    if 'values' not in r:
        time.sleep(90)
    for c in r['values']:
        e = r['values'][c]['expires']
        limit = r['values'][c]['credit_limit']
        if min_expires(e, min) == e:
            min = e
            resul['client'] = client
            resul['card'] = c
            resul['min_expires'] = e
            resul['limit'] = limit
        elif min_expires(e, min) == 'empate' and resul['limit'] > limit:
            resul['client'] = client
            resul['card'] = c
            resul['min_expires'] = e
            resul['limit'] = limit
    cont += 1
resul


In [58]:
import requests
import time

resul = {}
client_ids = df['client_id'].unique()
cont = 0

for client in client_ids:
    print(cont)
    r = requests.get('https://faas-lon1-917a94a7.doserverless.co/api/v1/web/fn-a1f52b59-3551-477f-b8f3-de612fbf2769/default/cards-data', params = {'client_id': client}).json()
    if 'values' not in r:
        print('esperando')
        time.sleep(120)
        r = requests.get('https://faas-lon1-917a94a7.doserverless.co/api/v1/web/fn-a1f52b59-3551-477f-b8f3-de612fbf2769/default/cards-data', params = {'client_id': client}).json()
        print(r)

    for card in r['values']:
        if len(resul) == 0:
            for col in r['values'][card]:
                resul[col] = []
            resul['client_id'] = []
            resul['card_id'] = []
        for col in r['values'][card]:
            resul[col].append(r['values'][card][col])
        resul['card_id'].append(card)
    cont += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [59]:
df_cards = pd.DataFrame()
for k in resul:
    df_cards[k] = resul[k]
df_cards

,acct_open_date,card_brand,card_number,card_on_dark_web,card_type,client_id,credit_limit,cvv,expires,has_chip,num_cards_issued,year_pin_last_changed,card_id
0,05/2008,Mastercard,5497590243197280,No,Debit (Prepaid),1556,$55,306,07/2022,YES,2,2008,2972
1,08/2010,Mastercard,5225389722526534,No,Debit,1556,$24772,891,09/2023,YES,2,2010,4333
2,04/2013,Mastercard,5918296616309874,No,Debit (Prepaid),561,$69,983,05/2024,YES,1,2013,2716
3,07/2006,Mastercard,5250625336912959,No,Debit,561,$26960,326,04/2022,YES,1,2010,3764
4,09/2005,Mastercard,5175842699412235,No,Credit,561,$9100,438,12/2024,YES,1,2015,4575
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4066,04/2016,Visa,4345309820081331,No,Credit,569,$5900,761,08/2022,YES,2,2016,2764
4067,06/2016,Mastercard,5046466769118356,No,Credit,616,$4500,561,05/2023,YES,2,2016,3611
4068,07/2016,Visa,4495571717001479,No,Credit,1862,$10200,778,09/2023,YES,2,2016,4049
4069,03/2017,Mastercard,5801798801399788,No,Credit,1527,$7100,527,06/2021,YES,2,2017,2366


In [60]:
df_cards.to_csv('../../data/raw/card_data.csv')

In [77]:
resul2 = {}
client_ids = df['client_id'].unique()
cont = 0

for client in client_ids:
    print(cont)
    r = requests.get('https://faas-lon1-917a94a7.doserverless.co/api/v1/web/fn-a1f52b59-3551-477f-b8f3-de612fbf2769/default/clients-data', params = {'client_id': client}).json()
    if 'values' not in r:
        print('esperando')
        time.sleep(120)
        r = requests.get('https://faas-lon1-917a94a7.doserverless.co/api/v1/web/fn-a1f52b59-3551-477f-b8f3-de612fbf2769/default/clients-data', params = {'client_id': client}).json()

    for col in r['values']:
        if len(resul2) == 0:
            for col2 in r['values']:
                resul2[col2] = []
            resul2['client_id'] = []
        resul2[col].append(r['values'][col])
    resul2['client_id'].append(client)
    cont += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [78]:
df_clients = pd.DataFrame()
for k in resul2:
    print(k, len(resul2[k]))
df_clients

address 1219
birth_month 1219
birth_year 1219
credit_score 1219
current_age 1219
gender 1219
latitude 1219
longitude 1219
num_credit_cards 1219
per_capita_income 1219
retirement_age 1219
total_debt 1219
yearly_income 1219
client_id 1219


""


In [79]:
df_clients = pd.DataFrame()
for k in resul2:
    df_clients[k] = resul2[k]
df_clients

,address,birth_month,birth_year,credit_score,current_age,gender,latitude,longitude,num_credit_cards,per_capita_income,retirement_age,total_debt,yearly_income,client_id
0,594 Mountain View Street,7,1989,740,30,Female,46.8,-100.76,4,$23679,67,$110153,$48277,1556
1,604 Pine Street,6,1971,834,48,Male,40.8,-91.12,5,$18076,67,$112139,$36853,561
2,2379 Forest Lane,4,1970,686,49,Male,33.18,-117.29,3,$16894,65,$36540,$34449,1129
3,903 Hill Boulevard,5,1967,685,52,Female,41.42,-87.35,5,$26168,67,$128676,$53350,430
4,166 River Drive,5,1968,711,51,Male,38.86,-76.6,2,$33529,69,$96182,$68362,848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,29 Martin Luther King Lane,2,1968,662,52,Male,34.83,-92.2,1,$22377,73,$81136,$45625,569
1215,4173 Wessex Avenue,5,1974,586,45,Female,32.19,-88.05,1,$17076,72,$63313,$34817,616
1216,7111 Lexington Avenue,4,1994,686,25,Female,41.57,-93.61,1,$18568,65,$57052,$37864,1862
1217,1426 Fifth Avenue,11,1953,578,66,Female,43.32,-71.18,1,$24779,67,$98763,$50523,1527


In [82]:
df_clients.to_csv('../../data/raw/client_data.csv')